# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


По столбцу 'children' минимальное значение -1
По столбцу 'days_employed' выявлены отрицательные значения (для минимума) и  максимум 401755.400475 дней (более 1000 лет в годах)

### Шаг 2.1 Заполнение пропусков

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


по столбцам days_employed и total_income зафиксированы пустые значения
Предполагаю, что там 0, так как отстутствие стажа означает и отсутствие дохода
заменним медианными значениями

In [3]:
data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [4]:
median_days_employed = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(value=median_days_employed)
median_total_income = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(value=median_total_income)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [5]:
#заменим количество детей со значения -1 на 0
data['children'] = data['children'].replace(-1, 0)
data.dropna(subset=['gender'])
data['children'] = abs(data['children'])
data['days_employed'] = abs(data['days_employed'])
data['dob_years'] = abs(data['dob_years'])
data['total_income'] = abs(data['total_income'])
data.describe()


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.541092,60277.957929,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.379943,133301.583103,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.608174,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,1808.053434,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [6]:
data['children'].abs()

0        1
1        1
2        0
3        3
4        0
        ..
21520    1
21521    0
21522    1
21523    3
21524    2
Name: children, Length: 21525, dtype: int64

### Шаг 2.3. Изменение типов данных.

In [7]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [8]:
#data['purpose'].value_counts()
data['education'] = data['education'].str.lower()


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [9]:
education_log = data[['education_id', 'education']]
education_log.drop_duplicates().reset_index(drop=True)
family_status_log = data[['family_status_id', 'family_status']]
family_status_log.drop_duplicates().reset_index(drop=True)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


### Шаг 2.6. Категоризация дохода.

In [10]:
def total_income_category(total_income):
    if total_income <=30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    return 'A'
data['total_income_category'] = data['total_income'].apply(total_income_category)


### Шаг 2.7. Категоризация целей кредита.

In [11]:
data['purpose'].value_counts()
def purpose_category(purpose):
    if purpose == 'свадьба':
        return 'проведение свадьбы'
    if purpose == 'на проведение свадьбы':
        return 'проведение свадьбы'
    if purpose == 'сыграть свадьбу':
        return 'проведение свадьбы'
    if purpose == 'операции с недвижимостью':
        return 'операции с недвижимостью'
    if purpose == 'покупка коммерческой недвижимости':
        return 'операции с недвижимостью'
    if purpose == 'операции с жильем':
        return 'операции с недвижимостью'
    if purpose == 'покупка жилья для сдачи':
        return 'операции с недвижимостью'
    if purpose == 'операции с коммерческой недвижимостью':
        return 'операции с недвижимостью'
    if purpose == 'покупка жилья':
        return 'операции с недвижимостью'
    if purpose == 'жилье':
        return 'операции с недвижимостью'
    if purpose == 'покупка жилья для семьи':
        return 'операции с недвижимостью'
    if purpose == 'строительство собственной недвижимости':
        return 'операции с недвижимостью'
    if purpose == 'недвижимость':
        return 'операции с недвижимостью'
    if purpose == 'операции со своей недвижимостью':
        return 'операции с недвижимостью'
    if purpose == 'строительство жилой недвижимости':
        return 'операции с недвижимостью'
    if purpose == 'покупка недвижимости':
        return 'операции с недвижимостью'
    if purpose == 'покупка своего жилья':
        return 'операции с недвижимостью'
    if purpose == 'строительство недвижимости':
        return 'операции с недвижимостью'
    if purpose == 'ремонт жилью':
        return 'операции с недвижимостью'
    if purpose == 'покупка жилой недвижимости':
        return 'операции с недвижимостью'
    if purpose == 'на покупку своего автомобиля ':
        return 'операции с автомобилем'
    if purpose == 'автомобиль':
        return 'операции с автомобилем'
    if purpose == 'сделка с подержанным автомобилем':
        return 'операции с автомобилем'
    if purpose == 'свой автомобиль':
        return 'операции с автомобилем'
    if purpose == 'на покупку подержанного автомобиля':
        return 'операции с автомобилем'
    if purpose == 'автомобили':
        return 'операции с автомобилем'
    if purpose == 'на покупку автомобиля':
        return 'операции с автомобилем'
    if purpose == 'приобретение автомобиля':
        return 'операции с автомобилем'
    if purpose == 'сделка с автомобилем':
        return 'операции с автомобилем'
    if purpose == 'заняться высшим образованием':
        return 'получение образования'
    if purpose == 'дополнительное образование':
        return 'получение образования'
    if purpose == 'высшее образование':
        return 'получение образования'
    if purpose == 'образование':
        return 'получение образования'
    if purpose == 'получение дополнительного образования':
        return 'получение образования'
    if purpose == 'получение образования':
        return 'получение образования'
    if purpose == 'профильное образование':
        return 'получение образования'
    if purpose == 'получение высшего образования':
        return 'получение образования'
    if purpose == 'заняться образованием':
        return 'получение образования'
data['purpose_category'] = data['purpose'].apply(purpose_category)
data


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,None


In [12]:
def children_category(children):
    if children == 0:
        return 'нет детей'
    else:
        return 'есть дети'
data['children_category'] = data['children'].apply(children_category)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,есть дети
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,есть дети
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,есть дети
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью,есть дети
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем,нет детей
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью,есть дети
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,None,есть дети


### Ответы на вопросы.

1. Есть ли зависимость между количеством детей и возвратом кредита в срок?1 1 

In [13]:
data.groupby('children_category')['debt'].mean().to_frame().sort_values(by='debt')

,debt
children_category,
нет детей,0.074951
есть дети,0.092373


Вывод: при наличии детей, просрочка выплат выше

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [14]:
data.groupby('family_status')['debt'].mean().to_frame().sort_values(by='debt')

,debt
family_status,
вдовец / вдова,0.065625
в разводе,0.071130
женат / замужем,0.075202
гражданский брак,0.092890
Не женат / не замужем,0.097405


Вывод: люди находящиеся в зарегистрированном или гражданском браке чаще допускают просрочку 

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [15]:
data.groupby('total_income_category')['debt'].mean().to_frame().sort_values(by='debt')

,debt
total_income_category,
D,0.060000
B,0.070621
A,0.080000
C,0.084540
E,0.090909


Вывод: категория людей с доходом "Е" наиболее чаще допускают просрочку

4. Как разные цели кредита влияют на его возврат в срок?

In [16]:
data.groupby('purpose_category')['debt'].mean().to_frame().sort_values(by='debt')

,debt
purpose_category,
операции с недвижимостью,0.072140
проведение свадьбы,0.079216
получение образования,0.091994
операции с автомобилем,0.093701


Вывод: Люди, которые взяли заем для операций, связанных с автомобилем, чаще допускают просрочку по кредитам

## Общий вывод:

Наиболее допускают просрочку люди с детьми и находящиеся в браке.
Самые ответственные люди с категорией доход "D" и с целью на проведение операций с недвижимостью 